In [1]:
import pandas as pd

pd.set_option("display.max_columns", None)

### Leitura dos arquivos

In [2]:
base_path = "olist/"

customers = pd.read_csv(base_path + "olist_customers_dataset.csv")
orders = pd.read_csv(base_path + "olist_orders_dataset.csv")
order_items = pd.read_csv(base_path + "olist_order_items_dataset.csv")
payments = pd.read_csv(base_path + "olist_order_payments_dataset.csv")
reviews = pd.read_csv(base_path + "olist_order_reviews_dataset.csv")
products = pd.read_csv(base_path + "olist_products_dataset.csv")
sellers = pd.read_csv(base_path + "olist_sellers_dataset.csv")
geolocation = pd.read_csv(base_path + "olist_geolocation_dataset.csv")
category_translation = pd.read_csv(base_path + "product_category_name_translation.csv")

### Padronização de nomes de colunas

In [3]:
def normalize_columns(df):
    df.columns = (
        df.columns
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
    )
    return df

dataframes_list = [
    customers, orders, order_items, payments,
    reviews, products, sellers, geolocation, category_translation
]

for df in dataframes_list:
    normalize_columns(df)

### Verificação de valores nulos

In [4]:
dataframes = {
    "customers": customers,
    "orders": orders,
    "order_items": order_items,
    "payments": payments,
    "reviews": reviews,
    "products": products,
    "sellers": sellers,
    "geolocation": geolocation,
    "category_translation": category_translation
}

def null_report(df, table_name):
    report = (
        df.isnull()
        .sum()
        .reset_index()
        .rename(columns={"index": "coluna", 0: "nulos"})
        .query("nulos > 0")
        .sort_values("nulos", ascending=False)
    )
    
    if not report.empty:
        report["tabela"] = table_name
    
    return report

def relatorio_dados_nulos(dfs: dict):
    relatorios = []

    for table_name, df in dfs.items():
        report = null_report(df, table_name)
        if not report.empty:
            relatorios.append(report)

    if relatorios:
        return pd.concat(relatorios, ignore_index=True)
    else:
        return pd.DataFrame(columns=["tabela", "coluna", "nulos"])

relatorio_nulos = relatorio_dados_nulos(dataframes)
relatorio_nulos

,coluna,nulos,tabela
0,order_delivered_customer_date,2965,orders
1,order_delivered_carrier_date,1783,orders
2,order_approved_at,160,orders
3,review_comment_title,87656,reviews
4,review_comment_message,58247,reviews
5,product_category_name,610,products
6,product_name_lenght,610,products
7,product_description_lenght,610,products
8,product_photos_qty,610,products
9,product_weight_g,2,products


#### Aqui vamos manter os dados nulos porque eles acabam se tornando informações úteis na nossa análise nesse caso e podemos filtrar depois com o SQL ignorando esses valores

### Ajustes de tipos de dados (DATAS)

In [5]:
orders.info()

<class 'pandas.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   order_id                       99441 non-null  str  
 1   customer_id                    99441 non-null  str  
 2   order_status                   99441 non-null  str  
 3   order_purchase_timestamp       99441 non-null  str  
 4   order_approved_at              99281 non-null  str  
 5   order_delivered_carrier_date   97658 non-null  str  
 6   order_delivered_customer_date  96476 non-null  str  
 7   order_estimated_delivery_date  99441 non-null  str  
dtypes: str(8)
memory usage: 6.1 MB


In [6]:
reviews.info()

<class 'pandas.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   review_id                99224 non-null  str  
 1   order_id                 99224 non-null  str  
 2   review_score             99224 non-null  int64
 3   review_comment_title     11568 non-null  str  
 4   review_comment_message   40977 non-null  str  
 5   review_creation_date     99224 non-null  str  
 6   review_answer_timestamp  99224 non-null  str  
dtypes: int64(1), str(6)
memory usage: 5.3 MB


In [7]:
order_date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in order_date_cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")

review_date_cols = [
    "review_creation_date",
    "review_answer_timestamp"
]

for col in review_date_cols:
    reviews[col] = pd.to_datetime(reviews[col], errors="coerce")

orders.info()

<class 'pandas.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  str           
 1   customer_id                    99441 non-null  str           
 2   order_status                   99441 non-null  str           
 3   order_purchase_timestamp       99441 non-null  datetime64[us]
 4   order_approved_at              99281 non-null  datetime64[us]
 5   order_delivered_carrier_date   97658 non-null  datetime64[us]
 6   order_delivered_customer_date  96476 non-null  datetime64[us]
 7   order_estimated_delivery_date  99441 non-null  datetime64[us]
dtypes: datetime64[us](5), str(3)
memory usage: 6.1 MB


### Verificação de duplicidades

In [9]:
def check_duplicated_rows(df):
    total = len(df)
    duplicated = df.duplicated().sum()
    
    return {
        "total_linhas": total,
        "linhas_duplicadas": duplicated,
        "percentual": round((duplicated / total) * 100, 4)
    }

for nome, df in dataframes.items():
    print(nome, check_duplicated_rows(df))

customers {'total_linhas': 99441, 'linhas_duplicadas': np.int64(0), 'percentual': np.float64(0.0)}
orders {'total_linhas': 99441, 'linhas_duplicadas': np.int64(0), 'percentual': np.float64(0.0)}
order_items {'total_linhas': 112650, 'linhas_duplicadas': np.int64(0), 'percentual': np.float64(0.0)}
payments {'total_linhas': 103886, 'linhas_duplicadas': np.int64(0), 'percentual': np.float64(0.0)}
reviews {'total_linhas': 99224, 'linhas_duplicadas': np.int64(0), 'percentual': np.float64(0.0)}
products {'total_linhas': 32951, 'linhas_duplicadas': np.int64(0), 'percentual': np.float64(0.0)}
sellers {'total_linhas': 3095, 'linhas_duplicadas': np.int64(0), 'percentual': np.float64(0.0)}
geolocation {'total_linhas': 1000163, 'linhas_duplicadas': np.int64(261831), 'percentual': np.float64(26.1788)}
category_translation {'total_linhas': 71, 'linhas_duplicadas': np.int64(0), 'percentual': np.float64(0.0)}


#### Aqui podemos notar que o geolocation existem linhas repetidas, mas essas duplicatas não são erros e sim são redundância proposital
#### Como essa tabela representa Mapeamentos de CEP para coordenadas e é esperado que o mesmo CEP apareça várias vezes. Isso não significa que pode atrapalhar na nossa análise. 

#### Conexão com o banco de dados

In [10]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

user = "postgres"
senha = "o5O$RD1,JWHMjF@n"
senha_codificada = quote_plus(senha)

banco_de_dados = "postgres"
host = "localhost"
porta = 5432

conexao = create_engine(
    f"postgresql+psycopg2://{user}:{senha_codificada}@{host}:{porta}/{banco_de_dados}"
)

#### Enviando os dados para o banco

In [11]:
def load_tables_to_postgres(
    conexao,
    dataframes: dict,
    schema="public",
    if_exists="replace",
    chunksize=10000
):
    for table_name, df in dataframes.items():
        print(f"Carregando tabela: {table_name}")

        df.to_sql(
            name=table_name,
            con=conexao,
            schema=schema,
            if_exists=if_exists,
            index=False,
            chunksize=chunksize,
            method="multi"
        )

        print(f"{table_name} carregada com sucesso\n")

load_tables_to_postgres(conexao, dataframes)

with conexao.connect() as conn:
    print("Conectado com sucesso!")

Carregando tabela: customers
customers carregada com sucesso

Carregando tabela: orders
orders carregada com sucesso

Carregando tabela: order_items
order_items carregada com sucesso

Carregando tabela: payments
payments carregada com sucesso

Carregando tabela: reviews
reviews carregada com sucesso

Carregando tabela: products
products carregada com sucesso

Carregando tabela: sellers
sellers carregada com sucesso

Carregando tabela: geolocation
geolocation carregada com sucesso

Carregando tabela: category_translation
category_translation carregada com sucesso

Conectado com sucesso!
